# 去哪儿网站数据分析

# 国内景点分布

In [1]:
import pandas as pd
from pyecharts.charts import Bar,Boxplot,WordCloud,Page,Grid,Boxplot,Map,Geo,Tab
from pyecharts import options as opts
import pymysql

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# 链接数据库
connection = pymysql.connect(host='127.0.0.1',port=3306,db='qunar',user='root',password='380324')
cursor = connection.cursor()

In [3]:
sql = "select city,price from message where price>0"
cursor.execute(sql)
data = pd.DataFrame(cursor.fetchall())

In [4]:
# 31个省市的价格
prices = data.groupby(data[0]).mean()
bar = Bar()
bar.set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size=10,interval=0,rotate=75)),\
                    title_opts=opts.TitleOpts(title="去哪儿网数据分析", subtitle="景点平均价格"),\
                   toolbox_opts=opts.ToolboxOpts(),legend_opts=opts.LegendOpts(is_show=False))
bar.add_yaxis("平均价格",[round(x,2) for x in list(prices[1])]).add_xaxis(list(prices.index),)
bar.set_global_opts()
bar.render_notebook()

西藏怎么这么贵，可能这就是旅游城市吧，那每个省份选取50*5个数据的结果呢？

In [5]:
# 32个省市的价格
bar = Bar()
bar.set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size=10,interval=0,rotate=75)))
bar.set_global_opts(title_opts=opts.TitleOpts(title="去哪儿网景点价格分析", subtitle="景点平均数"))
for i in range(0,150,50):
    price = []
    cities = []
    for city,prices in data.groupby(data[0]):
        price.append(round(prices[i:i+50][1].mean(),2))
        cities.append(city)
    bar.add_yaxis("前%s条"%(i+50),price)
    bar.set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="最大值"),
                    opts.MarkPointItem(type_="min", name="最小值"),
                    opts.MarkPointItem(type_="average", name="平均值"),
                ]
            )
    )
    bar.add_xaxis(cities)
bar.render_notebook()

西藏景点还真是贵

## 销量呢

In [8]:
sql = "select city,sale from message where sale>0"
cursor.execute(sql)
data = pd.DataFrame(cursor.fetchall())
data = data.astype({1:"double"})

In [11]:
# 31个省市的销量
sale = data.groupby(data[0]).mean()
bar = Bar()
bar.set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size=10,interval=0,rotate=75)))
bar.add_yaxis("平均销量",[round(x,2) for x in list(sale[1])])
bar.add_xaxis(list(sale.index))
bar.set_global_opts(title_opts=opts.TitleOpts(title="去哪儿网数据分析", subtitle="每省份平均销量"))
bar.render_notebook()

In [ ]:
那每个省份选取50*5个数据的结果呢

In [12]:
# 32个省市的价格
bar = Bar()
bar.set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size=10,interval=0,rotate=75)))
bar.set_global_opts(title_opts=opts.TitleOpts(title="去哪儿网景点价格分析", subtitle="景点平均数"))
for i in range(0,150,50):
    price = []
    cities = []
    for city,prices in data.groupby(data[0]):
        price.append(round(prices[i:i+50][1].mean(),2))
        cities.append(city)
    bar.add_yaxis("前%s条"%(i+50),price)
    bar.set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="最大值"),
                    opts.MarkPointItem(type_="min", name="最小值"),
                    opts.MarkPointItem(type_="average", name="平均值"),
                ]
            )
    )
    bar.add_xaxis(cities)
bar.add_yaxis("地区最大销量",[float(data) for data in data.groupby(data[0]).max().values])
bar.add_xaxis(cities)
bar.render_notebook()

### 将价格和销量合并在一起来看（只关注有销量和有价格的）

In [77]:
sql = "select city,price,sale from message where price>=1 and sale>=1"
cursor.execute(sql)
data = pd.DataFrame(cursor.fetchall())
data = data.astype({2:"double"})

In [78]:
sale_price = data.groupby(data[0]).mean()
bar = (
    Bar()
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size=10,interval=0,rotate=75)),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        title_opts=opts.TitleOpts(title="去哪儿网数据分析"),
        toolbox_opts=opts.ToolboxOpts(),
        datazoom_opts=opts.DataZoomOpts(type_="inside")
    )
    .add_xaxis(list(sale_price.index))
    .add_yaxis("平均价格",[round(x,2) for x in list(sale_price[1])],yaxis_index=0,color="#675bba")
    .add_yaxis("平均销量",[round(x,2) for x in list(sale_price[2])],yaxis_index=1,color="#34d3f3")
#     .extend_axis(
#         yaxis=opts.AxisOpts(
#             type_="value",
#             name="平均价格",
#             min_=0,
#             max_=600,
#             position="left",
#             axisline_opts=opts.AxisLineOpts(
#                 linestyle_opts=opts.LineStyleOpts(color="#675bba")
#             ),
#             axislabel_opts=opts.LabelOpts(formatter="{value} 元"),
#             splitline_opts=opts.SplitLineOpts(
#                 is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=1)
#             ),
#         )
#     )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .extend_axis(
        yaxis=opts.AxisOpts(
            type_="value",
            name="平均销量",
            min_=0,
            max_=600,
            position="right",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="#34d3f3")
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value} 次"),
        )
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
bar.render_notebook()

In [159]:
sql = """
SELECT m1.city,m1.price
FROM message m1
WHERE 50>(
    SELECT COUNT(1)
    FROM message m2
    WHERE m1.`city`=m2.`city` AND m1.`price`<m2.`price`
    ORDER BY price
    )
ORDER BY m1.city,m1.price;
"""
cursor.execute(sql)
price = pd.DataFrame(cursor.fetchall())

sql="""
SELECT city,sale
FROM message;
"""
cursor.execute(sql)
sale = pd.DataFrame(cursor.fetchall(),columns=["城市","销量"])
sale = sale.astype({"销量":"double"})

In [ ]:
## 取销量最高的50组价格和销量对比

In [198]:
box = (
    Boxplot()
    .add_xaxis([x for x,y in price.groupby(data[0])[1]])
    .add_yaxis("平均价格",box.prepare_data([list(y.values) for x,y in price.groupby(data[0])[1]]))
    .add_yaxis("平均销量",box.prepare_data([list(y[:50].values) for x,y in sale.groupby(sale["城市"])["销量"]]))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="销量价格箱型图"),
        datazoom_opts=opts.DataZoomOpts(type_="inside"),
        xaxis_opts = opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size=10,interval=0,rotate=75))
    )
)
box.render_notebook()

In [77]:
sql = "select city,price,sale from message where sale>0 or price>0"
cursor.execute(sql)
data = pd.DataFrame(cursor.fetchall(),columns=["省市","单价","销量"])
data = data.astype({"销量":"double"},copy=True)

In [113]:
price = data["单价"].groupby(data["省市"]).mean()
sale = data["销量"].groupby(data["省市"]).mean()

In [111]:
def change(lis):
    if lis[0] == "广西壮族自":
        lis[0]="广西"
    elif lis[0] == "内蒙古自治区":
        lis[0] = "内蒙古"
    elif lis[0][-1] != "区":
        lis[0] = lis[0][:-1]
    else:
        lis[0] = lis[0][:2]
    lis[1] = round(lis[1],2)
    return lis

In [118]:
map = (
    Map()
    .add(
        series_name = "平均价格",
        data_pair = [change(list(x)) for x in zip(price.index,price.values)]
    )
    .add(
    series_name = "平均销量",
    data_pair = [change(list(x)) for x in zip(sale.index,sale.values)]
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各省市平均价格—销量"),
        visualmap_opts=opts.VisualMapOpts(max_=200)
    )
)
map.render_notebook()

In [119]:
sql = "select city,note from message where note is not null"
cursor.execute(sql)
data = pd.DataFrame(cursor.fetchall())

In [121]:
# 统计词频，使用jieba分词库
import jieba
import re
# 处理标点符号
opts.ToolboxOpts(is_show=True)
charts = Page("去哪儿网评论词云",layout=Page.SimplePageLayout)
for city,message in data.groupby(data[0])[1]:
    va_count = {}
    for value in message.values:
        value = re.sub("[\s+\.【】。、》《\!\/_：\d\",$%^*(-+\"\'；“”‘’+|[+=！，。？、~@#￥%……&*（）]+", "",value)
        for z in jieba.lcut(value):
            if z not in ["的",""," ","一个","一","我","你","他","了","只","让","它","从","似","","和","为",\
                     "及","不","吧","很","使","有","在","以","去","级","带","是","与","内","着","之","才",\
                        "能","给","都","还","来","此","于","位于","一座","之一","而","处","能","这","还有","里",\
                         "该","并","其"] and not isinstance(z,int):
                va_count[z] = va_count.get(z,0)+1
    word = list(zip(va_count.keys(),va_count.values()))
    pic = WordCloud(opts.InitOpts(width="300px",height="200px")).\
    add(series_name="%s词云"%city,data_pair=word,shape = "diamond",word_size_range=[5,30]).\
    set_global_opts(title_opts=opts.TitleOpts(title="%s词云"%city))
    charts.add(pic)
charts.render()

'E:\\python\\render.html'

# 景点数地图

In [576]:
sql = "select place,city from message where place is not null and city is not null"
cursor.execute(sql)
data = pd.DataFrame(cursor.fetchall(),columns=["地区","省份"])

In [584]:
view = data.groupby(data["省份"])

In [786]:
tab = Tab() 
for x,y in view:
    city_dict = {}
    if x[-3:] == "自治区" or x == "广西壮族自":
        continue
    for city in y["地区"].values:
        if x not in ["上海市","北京市","重庆市","天津市"]:
            try:
                area = re.search("·(.*)·",city).group(1)
            except AttributeError:
                area = re.search("·(.*)",city).group(1)
        else:
            try:
                area = re.search(".*·.*·(.*)",city).group(1)
            except AttributeError:
                pass
        if area[-1:]!="区" and area[-1:]!="县":
            area = area+"市"
        city_dict[area] = city_dict.get(area,0)+1
#     print(city_dict)
    map = (
    Map()
    .add(
        series_name = "地图",
        data_pair = list(zip(city_dict.keys(),city_dict.values())),
        maptype = x[:-1],
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title=x),
                    visualmap_opts=opts.VisualMapOpts())
    )
    tab.add(map,x)
tab.render_notebook()